# Examples of scheduled circuit  (circuit with duration)

## Current status
1. [x] Adding a Delay instruction for circuits
2. [x] Two scheduling passes for implementing ALAP and ASAP by inserting Delays on the DAGCircuit.
```
from qiskit.transpiler.passes.scheduling import ASAPSchedule, ALAPSchedule
dag_with_delays = ALAPSchedule(backend_properties).run(dag)
```
3. [x] A simple scheduled_circuit.draw() to visualize timed blocks on the qubits
4. [x] Another scheduler that approximates the schedule by timesteps.
5. [x] Assembly to Qobj.
```
For qobj instructions something like:
{"name": "delay", "qubits": [i], "params": [length], "duration": length}  # "duration" may be optional
{"name": "cx", "qubits": [i,j], "duration": 320}  # "duration" may be optional
and
{"name": "timestep", "qubits": [0,...,N-1], "params": [length], "duration": 0}  # "duration" may be optional
{"name": "barrier", "qubits": [0,...,N-1], "duration": 0}  # "duration" may be optional
the lengths would have to be relative to a dt parameter in the noise model 
```

In [1]:
from qiskit import QuantumCircuit
from qiskit.test.mock.backends import FakeParis
backend = FakeParis()
print(f"dt={backend.configuration().dt}")

dt=2.2222222222222221e-10


## Manual construction of T2 experiment

In [2]:
qc = QuantumCircuit(1, 1, name="t2_experiment")
qc.h(0)
qc.delay(1000, 0, unit='ns')
qc.h(0)
qc.measure(0, 0)
qc.draw()

┌───┐┌──────────────┐┌───┐┌─┐
q_0: ┤ H ├┤ DELAY(1e-06) ├┤ H ├┤M├
     └───┘└──────────────┘└───┘└╥┘
c_0: ═══════════════════════════╩═

In [3]:
# 1000 ns = 4500 dt in FakeParis backend
print(qc.schedule(backend))

     |--------------------------------------------------------------------|
q_0: | U2(0,pi)[160] | DELAY(4500)[4500] | U2(0,pi)[160] | MEASURE[19200] |
     |--------------------------------------------------------------------|


## Schedule circuits with scheduling passes

In [4]:
from qiskit import transpile
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.transpiler.passes.scheduling.asap import ASAPSchedule
from qiskit.transpiler.passes.scheduling.alap import ALAPSchedule

In [5]:
qc = QuantumCircuit(2, name="bell_with_delay")
qc.h(0)
qc.delay(1000, 1, unit='ns')
qc.cx(0,1)
qc.draw()

┌───┐           
q_0: ─────┤ H ├────────■──
     ┌────┴───┴─────┐┌─┴─┐
q_1: ┤ DELAY(1e-06) ├┤ X ├
     └──────────────┘└───┘

In [6]:
qc.schedule(backend).draw()

     |---------------------------------------------------|
q_0: | DELAY(4340)[4340] | U2(0,pi)[160] | CX(0,1)[1408] |
q_1: |         DELAY(4500)[4500]         | CX(0,1)[1408] |
     |---------------------------------------------------|

`qc.schedule(backend)` is defined as follows.

In [7]:
transpiled = transpile(qc, backend=backend, optimization_level=0, basis_gates=['u1', 'u2', 'u3', 'cx', 'delay'])
dag = circuit_to_dag(transpiled)
dag_with_delays = ALAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

     |---------------------------------------------------|
q_0: | DELAY(4340)[4340] | U2(0,pi)[160] | CX(0,1)[1408] |
q_1: |         DELAY(4500)[4500]         | CX(0,1)[1408] |
     |---------------------------------------------------|


###  ASAP or ALAP Schedule

In [8]:
qc = QuantumCircuit(2, name="h2")
qc.h(0)
qc.x(1)
dag = circuit_to_dag(transpile(qc, backend=backend, optimization_level=0))
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ X ├
     └───┘

In [9]:
#ASAP
dag_with_delays = ASAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

     |---------------------------------|
q_0: | U2(0,pi)[160] | DELAY(160)[160] |
q_1: |         U3(pi,0,pi)[320]        |
     |---------------------------------|


In [10]:
#ALAP
dag_with_delays = ALAPSchedule(backend).run(dag)
scheduled = dag_to_circuit(dag_with_delays)
print(scheduled)

     |---------------------------------|
q_0: | DELAY(160)[160] | U2(0,pi)[160] |
q_1: |         U3(pi,0,pi)[320]        |
     |---------------------------------|


###  Schedule passes convert the duration units into dt (values are rounded)

In [11]:
bell = QuantumCircuit(2, name="bell_with_manual_delay_ns")
bell.h(0)
bell.delay(999, 1, unit='ns')
bell.cx(0,1)
print(bell.schedule(backend))

     |---------------------------------------------------|
q_0: | DELAY(4336)[4336] | U2(0,pi)[160] | CX(0,1)[1408] |
q_1: |         DELAY(4496)[4496]         | CX(0,1)[1408] |
     |---------------------------------------------------|


/Users/itoko/swd/qiskit-dev/qiskit-sdk-py/qiskit/transpiler/passes/scheduling/utils.py:55: UserWarning: Duration of delay is rounded to 4496 dt = 9.991111e-07 s from 9.990000e-07
  UserWarning)


In [12]:
bell = QuantumCircuit(2, name="bell_with_manual_delay_unitless")
bell.h(0)
bell.delay(999, 1)
bell.cx(0,1)
print(bell.schedule(backend))

     |-------------------------------------------------|
q_0: | DELAY(839)[839] | U2(0,pi)[160] | CX(0,1)[1408] |
q_1: |         DELAY(999)[999]         | CX(0,1)[1408] |
     |-------------------------------------------------|


##  Assembly qobj

In [13]:
qc = QuantumCircuit(2, name="bell_with_delay")
qc.h(0)
qc.delay(100, 1, unit='ns')
qc.cx(0,1)
qc.measure_all()
qc.draw()

┌───┐            ░ ┌─┐   
   q_0: ─────┤ H ├────────■───░─┤M├───
        ┌────┴───┴─────┐┌─┴─┐ ░ └╥┘┌─┐
   q_1: ┤ DELAY(1e-07) ├┤ X ├─░──╫─┤M├
        └──────────────┘└───┘ ░  ║ └╥┘
meas_0: ═════════════════════════╩══╬═
                                    ║ 
meas_1: ════════════════════════════╩═

In [14]:
sc = qc.schedule(backend)
sc.draw()

     |--------------------------------------------------------------------------
q_0: | DELAY(290)[290] | U2(0,pi)[160] | CX(0,1)[1408] | BARRIER(0,1)[0] | MEASU
q_1: |         DELAY(450)[450]         | CX(0,1)[1408] | BARRIER(0,1)[0] | MEASU
     |--------------------------------------------------------------------------
----------|
RE[19200] |
RE[19200] |
----------|

In [15]:
from qiskit import assemble
qobj = assemble(sc, backend=backend, shots=1000)
print("\n".join(str(qobj).split('\n')[-35:]))


	Instruction: delay
		params: [290]
		qubits: [0]
		duration: 290

	Instruction: u2
		params: [0, 3.141592653589793]
		qubits: [0]
		duration: 160

	Instruction: delay
		params: [450]
		qubits: [1]
		duration: 450

	Instruction: cx
		qubits: [0, 1]
		duration: 1408

	Instruction: barrier
		qubits: [0, 1]
		duration: 0

	Instruction: measure
		qubits: [0]
		memory: [0]
		duration: 19200

	Instruction: measure
		qubits: [1]
		memory: [1]
		duration: 19200




In [16]:
transpiled = transpile(qc, backend=backend, optimization_level=0, basis_gates=['u1', 'u2', 'u3', 'cx', 'delay'])
qobj = assemble(transpiled, backend=backend, shots=1000)
print("\n".join(str(qobj).split('\n')[-25:]))

{'memory_slots': 2, 'n_qubits': 27}

	Instruction: u2
		params: [0, 3.141592653589793]
		qubits: [0]

	Instruction: delay
		params: [1.0000000000000001e-07]
		qubits: [1]

	Instruction: cx
		qubits: [0, 1]

	Instruction: barrier
		qubits: [0, 1]

	Instruction: measure
		qubits: [0]
		memory: [0]

	Instruction: measure
		qubits: [1]
		memory: [1]




##  Timesteps

In [17]:
from qiskit.transpiler.passes.scheduling.timestepsasap import TimestepsASAPSchedule
def timestep_schedule(qc, backend, extra_basis=None):
    basis_gates = ['u1', 'u2', 'u3', 'cx', 'delay']
    if extra_basis:
        basis_gates += extra_basis
    transpiled = transpile(qc, backend=backend, optimization_level=0, basis_gates=basis_gates)
    dag = circuit_to_dag(transpiled)
    dag_with_delays = TimestepsASAPSchedule(backend).run(dag)
    scheduled = dag_to_circuit(dag_with_delays)
    return scheduled

In [18]:
qc = QuantumCircuit(2, name="bell_with_delay")
qc.h(0)
qc.delay(100, 1, unit='ns')
qc.cx(0,1)
qc.measure_all()
qc.draw()

┌───┐            ░ ┌─┐   
   q_0: ─────┤ H ├────────■───░─┤M├───
        ┌────┴───┴─────┐┌─┴─┐ ░ └╥┘┌─┐
   q_1: ┤ DELAY(1e-07) ├┤ X ├─░──╫─┤M├
        └──────────────┘└───┘ ░  ║ └╥┘
meas_0: ═════════════════════════╩══╬═
                                    ║ 
meas_1: ════════════════════════════╩═

In [19]:
sc = timestep_schedule(qc, backend)
print(sc.draw(qubits=[0, 1]))

     |--------------------------------------------------------------------------
q_0: |  U2(0,pi)[160]  | TIMESTEP(160)[0] | DELAY(290)[290] | TIMESTEP(290)[0] |
q_1: | DELAY(160)[160] | TIMESTEP(160)[0] | DELAY(290)[290] | TIMESTEP(290)[0] |
     |--------------------------------------------------------------------------
--------------------------------------------------------------------------------
   CX(0,1)[1408]   | BARRIER(0,1)[0] | TIMESTEP(1408)[0] |    MEASURE[19200]   |
   CX(0,1)[1408]   | BARRIER(0,1)[0] | TIMESTEP(1408)[0] |    MEASURE[19200]   |
--------------------------------------------------------------------------------
--------------------|
 TIMESTEP(19200)[0] |
 TIMESTEP(19200)[0] |
--------------------|


In [20]:
qobj = assemble(sc, backend=backend, shots=1000)
print("\n".join(str(qobj).split('\n')))

QASM Qobj: 0939e3fd-f1ff-499a-a37e-2a5da488bfea:
Config: {'memory': False,
 'memory_slots': 2,
 'n_qubits': 27,
 'parameter_binds': [],
 'shots': 1000}
Header: {'backend_name': 'fake_paris', 'backend_version': '1.0.3'}
Experiments:

QASM Experiment:
Header:
{'clbit_labels': [['meas', 0], ['meas', 1]],
 'creg_sizes': [['meas', 2]],
 'memory_slots': 2,
 'n_qubits': 27,
 'name': 'bell_with_delay',
 'qreg_sizes': [['q', 27]],
 'qubit_labels': [['q', 0],
                  ['q', 1],
                  ['q', 2],
                  ['q', 3],
                  ['q', 4],
                  ['q', 5],
                  ['q', 6],
                  ['q', 7],
                  ['q', 8],
                  ['q', 9],
                  ['q', 10],
                  ['q', 11],
                  ['q', 12],
                  ['q', 13],
                  ['q', 14],
                  ['q', 15],
                  ['q', 16],
                  ['q', 17],
                  ['q', 18],
                  ['q', 19],
    

In [21]:
import numpy as np
from qiskit.extensions import UnitaryGate
U20 = UnitaryGate(np.eye(2), label="U20")
U20.duration = 20
U7 = UnitaryGate(np.eye(2), label="U7")
U7.duration = 7
qc = QuantumCircuit(2, name="U20-U7")
qc.append(U20, [0])
qc.append(U20, [0])
qc.append(U7, [1])
qc.append(U7, [1])
qc.append(U7, [1])
qc.append(U7, [1])
qc.append(U7, [1])
qc.append(U7, [1])
qc.draw()

┌─────┐┌─────┐                        
q_0: ┤ U20 ├┤ U20 ├────────────────────────
     └┬────┤└┬────┤┌────┐┌────┐┌────┐┌────┐
q_1: ─┤ U7 ├─┤ U7 ├┤ U7 ├┤ U7 ├┤ U7 ├┤ U7 ├
      └────┘ └────┘└────┘└────┘└────┘└────┘

In [22]:
sc = timestep_schedule(qc, backend, extra_basis=["unitary"])
print(sc.draw(qubits=[0, 1]))

     |--------------------------------------------------------------------------
q_0: |        U20        | TIMESTEP(20)[0] |        U20        | TIMESTEP(20)[0]
q_1: |U7|U7| DELAY(6)[6] | TIMESTEP(20)[0] |U7|U7| DELAY(6)[6] | TIMESTEP(20)[0]
     |--------------------------------------------------------------------------
---------------------------------------------------------------|
 | DELAY(7)[7] | TIMESTEP(7)[0] | DELAY(7)[7] | TIMESTEP(7)[0] |
 |      U7     | TIMESTEP(7)[0] |      U7     | TIMESTEP(7)[0] |
---------------------------------------------------------------|


### Idle qubits must be filled with delays, right?

```
      |-------------------------------------------------|
 q_0: | DELAY(839)[839] | U2(0,pi)[160] | CX(0,1)[1408] |
 q_1: |         DELAY(999)[999]         | CX(0,1)[1408] |
      |-------------------------------------------------|
```

```
      |-------------------------------------------------|
 q_0: | DELAY(839)[839] | U2(0,pi)[160] | CX(0,1)[1408] |
 q_1: |         DELAY(999)[999]         | CX(0,1)[1408] |
 q_2: |                DELAY(2407)[2407]                |
 q_3: |                DELAY(2407)[2407]                |
 q_4: |                DELAY(2407)[2407]                |
 q_5: |                DELAY(2407)[2407]                |
 ...
q_26: |                DELAY(2407)[2407]                |
      |-------------------------------------------------|
 ```

##  Issues identified

- Should we output delay when qasm()? Will delay be included in QASM3? (related to https://github.com/Qiskit/qiskit-terra/issues/4312)
- We cannot get length of measure from backend.properties (I struggled [here](https://github.com/itoko/qiskit-sdk-py/blob/1c7bc7f7c8299dab7abc9f0ebb3ef70e496267b3/qiskit/transpiler/passes/scheduling/asap.py#L38))
- As Lauren suggested, we need more useful helper function (or class) for getting duration (gate length) of each instruction.
- How to distinguish regular circuit and scheduled circuit in the code level: `if self.duration` or `if isinstance(qc, ScheduleCircuit)`

##  Questions

1. Can you give me any use case where operations of scheduled circuits are required? (When do we really need operations more than `append()`?)

## T2 experiment with dynamical decoupling?

In [23]:
N = 2

In [24]:
dd = QuantumCircuit(1, name="XYXY")
dd.x(0)
dd.y(0)
dd.x(0)
dd.y(0)
dd.draw()

┌───┐┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ Y ├┤ X ├┤ Y ├
     └───┘└───┘└───┘└───┘

In [25]:
t2dd = QuantumCircuit(1, 1, name="t2_with_dynamical_decoupling")
t2dd.h(0)
t2dd.append(dd.to_instruction().repeat(N), qargs=[0])
t2dd.h(0)
t2dd.measure(0, 0)
t2dd.draw()

┌───┐┌────────┐┌───┐┌─┐
q_0: ┤ H ├┤ XYXY*2 ├┤ H ├┤M├
     └───┘└────────┘└───┘└╥┘
c_0: ═════════════════════╩═

In [26]:
t2 = QuantumCircuit(1, 1, name="t2_with_delay")
t2.h(0)
t2.delay(N * dd.schedule(backend).duration)
t2.h(0)
t2.measure(0, 0)
t2.draw()

┌───┐┌─────────────┐┌───┐┌─┐
q_0: ┤ H ├┤ DELAY(2560) ├┤ H ├┤M├
     └───┘└─────────────┘└───┘└╥┘
c_0: ══════════════════════════╩═

In [27]:
t2dd.schedule(backend).draw(qubits=[0])

     |--------------------------------------------------------------------------
q_0: | U2(0,pi)[160] | U3(pi,0,pi)[320] | U3(pi,pi/2,pi/2)[320] | U3(pi,0,pi)[32
     |--------------------------------------------------------------------------
--------------------------------------------------------------------------------
0] | U3(pi,pi/2,pi/2)[320] | U3(pi,0,pi)[320] | U3(pi,pi/2,pi/2)[320] | U3(pi,0,
--------------------------------------------------------------------------------
------------------------------------------------------------------|
pi)[320] | U3(pi,pi/2,pi/2)[320] | U2(0,pi)[160] | MEASURE[19200] |
------------------------------------------------------------------|

In [28]:
t2.schedule(backend).draw(qubits=[0])

     |--------------------------------------------------------------------|
q_0: | U2(0,pi)[160] | DELAY(2560)[2560] | U2(0,pi)[160] | MEASURE[19200] |
     |--------------------------------------------------------------------|

##  Tests

In [29]:
ghz3 = QuantumCircuit(3, name="ghz3")
ghz3.h(0)
ghz3.cx(0,1)
ghz3.cx(1,2)
ghz3.schedule(backend).draw(qubits=[0,1,2])

     |-----------------------------------------------------|
q_0: |  U2(0,pi)[160]  | CX(0,1)[1408] | DELAY(1632)[1632] |
q_1: | DELAY(160)[160] | CX(0,1)[1408] |   CX(1,2)[1632]   |
q_2: |        DELAY(1568)[1568]        |   CX(1,2)[1632]   |
     |-----------------------------------------------------|

In [30]:
qc = QuantumCircuit(2, name="test_barrier")
qc.h(0)
qc.barrier()
qc.h(1)
qc.draw()
qc.schedule(backend).draw(qubits=[0,1])

     |-----------------------------------------------------|
q_0: |  U2(0,pi)[160]  | BARRIER(0,1)[0] | DELAY(160)[160] |
q_1: | DELAY(160)[160] | BARRIER(0,1)[0] |  U2(0,pi)[160]  |
     |-----------------------------------------------------|